In [46]:
import pandas as pd
import numpy as np

def svd_csv(file_path, rank):
    # Read the .csv file into a DataFrame
    X = pd.read_csv(file_path)
    
    # Store the "time" column
    time_column = X['time']
    
    # Store original column names (without 'time')
    original_columns = X.drop('time', axis=1).columns

    # Remove the "time" column
    X = X.drop('time', axis=1)

    # Transpose the DataFrame
    X = X.transpose()

    # Convert DataFrame to numpy array for SVD
    X_np = X.values

    # Implement SVD
    U, S, VT = np.linalg.svd(X_np, full_matrices=False)

    # Convert singular values to a diagonal matrix
    S = np.diag(S)

    # Check if rank is less than or equal to the number of singular values
    if rank <= S.shape[0]:
        # Construct approximate DataFrame
        Xapprox = pd.DataFrame(U[:,:rank] @ S[0:rank,:rank] @ VT[:rank,:])
    else:
        print(f'Rank={rank} is greater than the number of singular values.')
        return None
    
    # Transpose the DataFrame back to its original orientation
    Xapprox = Xapprox.transpose()
    
    # Assign back the original column names
    Xapprox.columns = original_columns
    
    # Add back the "time" column as the first column
    Xapprox.insert(0, 'time', time_column)

    # Return the approximation
    return Xapprox

In [47]:
def write_opensim_file(data, file_path, csv_file_path=None):
    if csv_file_path is not None:
        data = pd.read_csv(csv_file_path)
    
    nRows = len(data)
    nColumns = len(data.columns)

    header = f"""Coordinates
    version=1
    nRows={nRows}
    nColumns={nColumns}
    inDegrees=yes

    Units are S.I. units (second, meters, Newtons, ...)
    If the header above contains a line with 'inDegrees', this indicates whether rotational values are in degrees (yes) or radians (no).

    endheader
    """

    with open(file_path, 'w') as file:
        file.write(header)
    data.to_csv(file_path, sep='\t', mode='a', index=False, float_format='%.8f')

In [50]:
import os

def process_and_write(file_path, rank):
    # Extract the filename without extension from the file path
    base_name = os.path.basename(file_path)
    filename_without_extension = os.path.splitext(base_name)[0]

    # Append rank to the filename
    new_filename = f"{filename_without_extension}_rank_{rank}.sto"

    # Process the CSV file
    approximation = svd_csv(file_path, rank)

    # Write the DataFrame to a new file
    write_opensim_file(approximation, new_filename)

In [75]:
file_path = 'Z:/Research Drive/Riehm/DDSE/SPARC_CODE_DATA/Prepare_UO1_Files/OPENSIM/Data/box1_enable_rotated_Kinematics_q.csv'  # Replace with your actual file path
rank = 50  # Replace with your desired rank
process_and_write(file_path, rank)